# ДЗ 1 Индекс

### os.listdir  
возвращает список файлов в данной директории

При обходе файлов не забывайте исключать системные директории, такие как .DS_Store

### os.walk
root - начальная директория  
dirs - список поддиректорий (папок)   
files - список файлов в этих поддиректориях  

### Пример реализации обратного индекса через CountVectorizer. Пользуйтесь.

In [35]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
 
# инициализируем
vectorizer = CountVectorizer(analyzer='word')

# составляем корпус документов
corpus = [
  'слово1 слово2 слово3',
  'слово2 слово3',
  'слово1 слово2 слово1',
  'слово4'
]

# считаем
X = vectorizer.fit_transform(corpus)
 
# получится следующая структура:
#        | слово1 | слово2 | слово3 | слово4
# текст1 |   1    |    1   |   1    |   0
# текст2 |   0    |    1   |   1    |   0
# текст3 |   2    |    1   |   0    |   0
# текст4 |   0    |    0   |   0    |   1

# показать матрицу
print('X:\n', X.toarray(), '\n')

 
# чтобы получить сгенерированный словарь, из приведенной структуры CountVectorizer
# порядок совпадает с матрицей
print('get_feature_names:', vectorizer.get_feature_names(), '\n')  # ['слово1', 'слово2', 'слово3', 'слово4']
 
    
# чтобы узнать индекс токена в словаре
print('vocabulary_.get:', vectorizer.vocabulary_.get('слово1')) # вернет 0
print('vocabulary_.get:', vectorizer.vocabulary_.get('слово4'), '\n') # вернет 3

 
# теперь можно быстро подсчитать вектор для нового документа
print('transform:', vectorizer.transform(['слово1 слово4 слово4']).toarray())  # результат [[1 0 0 2]]
print('transform:', vectorizer.transform(['слово5']).toarray(), '\n')

 
# чтобы узнать количественное вхождение каждого слова:
matrix_freq = np.asarray(X.sum(axis=0)).ravel()
print('matrix_freq:', matrix_freq, '\n')  # результат [3 3 2 1] 

final_matrix = np.array([np.array(vectorizer.get_feature_names()), matrix_freq])
print('final_matrix:', final_matrix, '\n')  # результат [['слово1' 'слово2' 'слово3' 'слово4'], ['3' '3' '2' '1']] 

X:
 [[1 1 1 0]
 [0 1 1 0]
 [2 1 0 0]
 [0 0 0 1]] 

get_feature_names: ['слово1', 'слово2', 'слово3', 'слово4'] 

vocabulary_.get: 0
vocabulary_.get: 3 

transform: [[1 0 0 2]]
transform: [[0 0 0 0]] 

matrix_freq: [3 3 2 1] 

final_matrix: [['слово1' 'слово2' 'слово3' 'слово4']
 ['3' '3' '2' '1']] 



In [36]:
X.toarray()

array([[1, 1, 1, 0],
       [0, 1, 1, 0],
       [2, 1, 0, 0],
       [0, 0, 0, 1]], dtype=int64)

In [37]:
vectorizer.transform(['слово1 слово4 слово4']).toarray()

array([[1, 0, 0, 2]], dtype=int64)

#  Индекс 

Сам по себе индекс - это просто формат хранения данных, он не может осуществлять поиск. Для этого необходимо добавить к нему определенную метрику. Это может быть что-то простое типа булева поиска, а может быть что-то более специфическое или кастомное под задачу.

Давайте посмотрим, что полезного можно вытащить из самого индекса.    
Например, индекс - это информация о частоте встречаемости слова в каждом документе.   
Из этого можно понять, например:
1. какое слово является самым часто употребимым / редким
2. какие слова встречаются всегда вместе - так можно парсить твиттер, fb, форумы и отлавливать новые устойчивые выражения в речи
3. как эти документы кластеризуются по N тематикам согласно словам, которые в них упоминаются 

## __Задача__: 

**Data:** Коллекция субтитров сезонов Друзей. Одна серия - один документ.

**To do:** 

**1 Создайте обратный индекс этой базы, используя CountVectorizer. 
То, что вы получите, называется матрица Term-Document.**

Компоненты вашей реализации:
    - Функция препроцессинга данных. Включите туда лемматизацию, приведение к одному регистру, удаление пунктуации и стоп-слов.
    - Функция индексирования данных. На выходе создает обратный индекс, он же матрица Term-Document.

**2 С помощью обратного индекса посчитайте:** 


a) какое слово является самым частотным

b) какое самым редким

c) какой набор слов есть во всех документах коллекции

d) кто из главных героев статистически самый популярный (упонимается чаще всего)? Имена героев:
- Моника
- Рэйчел 
- Чендлер
- Фиби
- Росс
- Джоуи, Джои

**На что направлены эти задачи:** 
1. Навык построения обратного индекса
2. Навык работы с этим индексом, а именно как с помощью numpy или pandas достать нужную информацию из матрицы данных

[download_friends_corpus](https://yadi.sk/d/4wmU7R8JL-k_RA?w=1)

In [38]:
import os

curr_dir = os.getcwd()
sub_dir = os.path.join(curr_dir, 'friends-data')

In [39]:
paths = []
for root, dirs, files in os.walk(curr_dir):
    for name in files:
        if name.endswith('txt'):
            paths.append(os.path.join(root, name))

In [40]:
import pymorphy2
from string import punctuation
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def preproc(path):
    morph = pymorphy2.MorphAnalyzer()
    with open(path, 'r', encoding='utf-8-sig') as f:
        text = f.read()
        text = [word.lower().strip(punctuation) for word in text.split()]
        text = [word for word in text if word not in stopwords.words('russian')]
        text = [word for word in text if word != '']
        del text[-4:] # meta info
    
    lemmas = str()
    known_words = {}

    for word in text:
        if word in known_words:
            lemmas += (known_words[word] + ' ')
        else:
            result = morph.parse(word)[0].normal_form
            lemmas += (result + ' ')
            known_words[word] = result
    
    return lemmas

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\trekc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [41]:
from tqdm import tqdm

texts = []
for path in tqdm(paths):
    processed = preproc(path)
    texts.append(processed)         

100%|████████████████████████████████████████████████████████████████████████████████| 165/165 [01:50<00:00,  1.49it/s]


In [64]:
vectorizer = CountVectorizer(analyzer='word')
X = vectorizer.fit_transform(texts).todense()

In [55]:
import pandas as pd

In [79]:
data = pd.DataFrame(X, columns = vectorizer.get_feature_names())
data

,00,000,007,009,02,038,03815,0fps,10,100,...,ёвить,ёй,ёкнуть,ёлка,ёлочный,ёпэрэсотэ,ёрл,ёрш,ёршик,ёще
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,0,0,7,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
161,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
162,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
163,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [176]:
summarizer = data.sum()

In [178]:
summarizer[summarizer == summarizer.max()]

это    6573
dtype: int64

In [183]:
summarizer[summarizer == summarizer.min()]

03815      1
0fps       1
101        1
102        1
110        1
          ..
ёлочный    1
ёрл        1
ёрш        1
ёршик      1
ёще        1
Length: 6866, dtype: int64

In [205]:
def get_mms(data):
    summarizer = data.sum()
    max_value = summarizer[summarizer == summarizer.max()]
    min_value = summarizer[summarizer == summarizer.min()]
    print(max_value)
    print(min_value)

In [206]:
get_mms(data)

это    6573
dtype: int64
03815      1
0fps       1
101        1
102        1
110        1
          ..
ёлочный    1
ёрл        1
ёрш        1
ёршик      1
ёще        1
Length: 6866, dtype: int64


In [120]:
inclusive = list()
for word, index in data.iteritems():
    temp = []
    for item in index:
        if item > 0:
            temp.append(word)
    if len(temp) == 165:
        inclusive.append(temp[0])

In [122]:
inclusive

['думать', 'знать', 'просто', 'сказать', 'ты', 'хотеть', 'это']

In [197]:
charac = {
    'Моника': 0,
    'Рэйчел': 0,
    'Чендлер': 0,
    'Фиби': 0,
    'Росс': 0,
    'Джоуи': 0
}


for index, numbers in data.iteritems():
    if index == 'моника' or index == 'мон':
        for number in numbers:
            charac['Моника'] += number

    if index == 'рэйчел' or index == 'рейч':
        for number in numbers:
            charac['Рэйчел'] += number            
            
    if index == 'чендлер' or index == 'чэндлер' or index == 'чен':
        for number in numbers:
            charac['Чендлер'] += number

    if index == 'фиби' or index == 'фибс':
        for number in numbers:
            charac['Фиби'] += number

    if index == 'росс':
        for number in numbers:
            charac['Росс'] += number

    if index == 'Джоуи' or index == 'джои' or index == 'джо':
        for number in numbers:
            charac['Джоуи'] += number
            


In [199]:
max(charac, key=charac.get), max(charac.values())

('Росс', 1016)